In [24]:
import json
from pathlib import Path
from tqdm import tqdm
from loguru import logger

In [25]:
logger.add("logs/{time}.log")

In [26]:
root_path = Path("results")
model_paths = [root_path / "gemini-pro-text", root_path / "gemini-pro-chat"]

In [27]:
for model_path in tqdm(model_paths):
    task_paths = [task for task in model_path.iterdir() if task.is_dir()]
    for task_path in tqdm(task_paths):
        pe_paths = [pe for pe in task_path.iterdir() if pe.is_dir()]
        for pe_path in tqdm(pe_paths):
            files = list(pe_path.glob("*.json"))
            files = [file for file in files if "summary" not in file.name]
            for file in tqdm(files):
                with open(file, "r") as f:
                    data = json.load(f)

                if "finish_reason: SAFETY" in data["response"] or "block_reason: SAFETY" in data["response"]:
                    logger.info(f"Correcting {file} with {data['response']}")
                    data["response"] = ""
                    
                    with open(file, "w") as f:
                        json.dump(data, f, indent=4)